In [559]:
import pandas as pd
import numpy as np
import pymc as pm
from sklearn.cluster import KMeans
import random
from sklearn.preprocessing import StandardScaler

In [394]:
flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")
flipped

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/3063244417.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  flipped = pd.read_csv("https://github.com/cnickol26/BigDataBowl2023/blob/main/data_flipped.csv?raw=true")


,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end,Flip
0,202109090097,41.56,23.92,2021090900,97,53441.0,6,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.12,-1.99,none,40.0,53441.0,42403.0,1
1,202109090097,41.56,23.92,2021090900,97,53441.0,7,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.06,-1.96,none,40.0,53441.0,42403.0,1
2,202109090097,41.56,23.92,2021090900,97,53441.0,8,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.96,-1.91,none,40.0,53441.0,42403.0,1
3,202109090097,41.56,23.92,2021090900,97,53441.0,9,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.80,-1.85,none,40.0,53441.0,42403.0,1
4,202109090097,41.56,23.92,2021090900,97,53441.0,10,2021-09-10 00:26:32,11.0,DAL,...,TRADITIONAL,0,6.0,1.56,-1.72,none,40.0,53441.0,42403.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374366,20211025003926,85.95,23.68,2021102500,3926,NaN,45,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,7.18,3.67,none,44.0,37097.0,46083.0,0
1374367,20211025003926,85.95,23.68,2021102500,3926,NaN,46,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,7.22,3.89,none,44.0,37097.0,46083.0,0
1374368,20211025003926,85.95,23.68,2021102500,3926,NaN,47,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,7.0,7.27,4.08,none,44.0,37097.0,46083.0,0
1374369,20211025003926,85.95,23.68,2021102500,3926,NaN,48,2021-10-26 03:14:09,NaN,football,...,TRADITIONAL,0,7.0,7.31,4.26,none,44.0,37097.0,46083.0,0


In [395]:
frames_per_play = flipped[flipped['pff_positionLinedUp']=='QB'].groupby('uniqueplayId', as_index=False).size().rename(columns={'size':'play_length_frames'})

In [396]:
flipped = flipped.merge(frames_per_play, on='uniqueplayId', how='left')

In [397]:
flipped['play_length_frames'].min()

16.0

In [398]:
flipped.columns

Index(['uniqueplayId', 'football_x', 'football_y', 'gameId', 'playId', 'nflId',
       'frameId', 'time', 'jerseyNumber', 'team', 'playDirection', 'x', 'y',
       's', 'a', 'dis', 'o', 'dir', 'event', 'pff_role', 'pff_positionLinedUp',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'dropBackType',
       'pff_playAction', 'ball_snap_frame', 'new_x', 'new_y', 'chip_side',
       'end_frame', 'Left_end', 'Right_end', 'Flip', 'play_length_frames'],
      dtype='object')

In [399]:
def name_func(pos):
    temp = {'nflId':pos+'_nflId',
     'jerseyNumber':pos+'_jerseyNumber',
     'team':pos+'_team', 
     'x':pos+'_x', 
     'y':pos+'_y',
     's':pos+'_s', 
     'a':pos+'_a', 
     'dis':pos+'_dis', 
     'o':pos+'_o', 
     'dir':pos+'_dir', 
     'pff_role':pos+'_pff_role', 
     'pff_positionLinedUp':pos+'_pff_positionLinedUp',
     'pff_nflIdBlockedPlayer':pos+'_pff_nflIdBlockedPlayer', 
     'pff_blockType':pos+'_pff_blockType', 
     'new_x':pos+'_new_x', 
     'new_y':pos+'_new_y', 
     'Flip':pos+'_Flip'}
    return temp

In [400]:
qb = flipped[(flipped['pff_positionLinedUp']=='QB')]
qb = qb.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('QB')).reset_index(drop=True)

In [401]:
qb_flip = flipped[(flipped['pff_positionLinedUp']=='QB_flip')]
qb_flip = qb_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('QB')).reset_index(drop=True)

In [402]:
ball = flipped[(flipped['pff_positionLinedUp']=='ball')]
ball = ball.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('ball')).reset_index(drop=True)

In [403]:
ball_flip = flipped[(flipped['pff_positionLinedUp']=='ball_flip')]
ball_flip = ball_flip.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('ball')).reset_index(drop=True)


In [404]:
rt = flipped[(flipped['pff_positionLinedUp']=='RT')]
rt = rt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('Tackle')).reset_index(drop=True)

In [405]:
lt = flipped[(flipped['pff_positionLinedUp']=='LT')]
lt = lt.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('Tackle')).reset_index(drop=True)

In [406]:
le = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==1)]
le = le.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('End')).reset_index(drop=True)

In [407]:
re = flipped[((flipped['pff_role']=='Pass Rush')|(flipped['pff_role']=='Coverage'))&
             (flipped['Flip']==0)]
re = re.drop(['football_x', 'football_y', 'gameId', 'playId',
       'time', 'playDirection', 'event', 'ball_snap_frame', 'end_frame', 
       'Right_end', 'Left_end', 'dropBackType', 'pff_playAction', 'play_length_frames',
       'chip_side'], axis=1).rename(columns=name_func('End')).reset_index(drop=True)

In [408]:
play = flipped[(flipped['pff_positionLinedUp']=='QB')][['uniqueplayId', 'football_x', 'football_y', 
                                                        'gameId', 'playId', 'frameId', 'time', 'playDirection', 
                                                        'event', 'ball_snap_frame', 'end_frame', 'play_length_frames',
                                                        'Right_end', 'Left_end']].reset_index(drop=True)

In [409]:
final_left = play.merge(qb, on=['uniqueplayId', 'frameId'], how='left').merge(ball, on=['uniqueplayId', 'frameId'], how='left')

In [410]:
final_left = final_left.merge(lt, on=['uniqueplayId', 'frameId'], how='left').merge(re, on=['uniqueplayId', 'frameId'], how='left')

In [411]:
final_right = play.merge(qb_flip, on=['uniqueplayId', 'frameId'], how='left').merge(ball_flip, on=['uniqueplayId', 'frameId'], how='left')

In [412]:
final_right = final_right.merge(rt, on=['uniqueplayId', 'frameId'], how='left').merge(le, on=['uniqueplayId', 'frameId'], how='left')

In [413]:
final = pd.concat([final_left, final_right])

In [414]:
play_ex = final[final['uniqueplayId']==20210919023392]

Before classifying:<br>
> - Remove frames before ball is snapped and after it is thrown<br>
> - Remove tackle and end where the end's pff_role is Coverage<br>
> - First check: Remove plays where end moves into the inside (stunt)<br>
> - Remove plays where the distance between end and tackle is large directly after ball snap (not against each other)<br>
<br>

### Classifying
##### Clustering into two groups<br>
Predictor variables:<br>
> - Percentage that tackle/end are looking at each other <br>
> - Percentage that Euclidian distance is less than 0.5 (0.75)<br>
> - Total distance traveled of defensive end<br>
> - Max speed during rush<br>

In [415]:
def dist(x_1, y_1, x_2, y_2):
    return np.sqrt(np.sum([(x_1-x_2)**2, (y_1-y_2)**2], axis=0))

In [416]:
final['tackle_end_dist'] = dist(final['Tackle_new_x'], final['Tackle_new_y'], final['End_new_x'], final['End_new_y'])

In [417]:
angle_range = 20
def check_angle(x, y):
    diff = abs(abs(x-y)-180)
    if diff <= angle_range:
        return 1
    else:
        return 0

In [418]:
final['tackle_end_facing'] = final.apply(lambda row: check_angle(row['Tackle_o'], row['End_o']),  axis=1)

In [439]:
final

,uniqueplayId,football_x,football_y,gameId,playId,frameId,time,playDirection,event,ball_snap_frame,...,End_dir,End_pff_role,End_pff_positionLinedUp,End_pff_nflIdBlockedPlayer,End_pff_blockType,End_new_x,End_new_y,End_Flip,tackle_end_dist,tackle_end_facing
0,202109090097,41.56,23.92,2021090900,97,6,2021-09-10 00:26:31,right,ball_snap,6.0,...,247.75,Pass Rush,ROLB,NaN,NaN,2.34,-8.71,0.0,6.104891,0
1,202109090097,41.56,23.92,2021090900,97,7,2021-09-10 00:26:31,right,None,6.0,...,252.79,Pass Rush,ROLB,NaN,NaN,2.27,-8.70,0.0,6.009076,0
2,202109090097,41.56,23.92,2021090900,97,8,2021-09-10 00:26:31,right,None,6.0,...,248.14,Pass Rush,ROLB,NaN,NaN,2.12,-8.62,0.0,5.785404,0
3,202109090097,41.56,23.92,2021090900,97,9,2021-09-10 00:26:31,right,None,6.0,...,246.73,Pass Rush,ROLB,NaN,NaN,1.93,-8.54,0.0,5.508049,0
4,202109090097,41.56,23.92,2021090900,97,10,2021-09-10 00:26:32,right,None,6.0,...,243.55,Pass Rush,ROLB,NaN,NaN,1.55,-8.32,0.0,5.089764,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168448,20211025003926,85.95,23.68,2021102500,3926,45,2021-10-26 03:14:08,left,None,7.0,...,26.59,Pass Rush,LOLB,NaN,NaN,4.83,-3.77,1.0,1.314002,0
168449,20211025003926,85.95,23.68,2021102500,3926,46,2021-10-26 03:14:08,left,None,7.0,...,23.83,Pass Rush,LOLB,NaN,NaN,4.91,-3.94,1.0,1.338096,0
168450,20211025003926,85.95,23.68,2021102500,3926,47,2021-10-26 03:14:09,left,None,7.0,...,21.57,Pass Rush,LOLB,NaN,NaN,4.98,-4.11,1.0,1.362424,0
168451,20211025003926,85.95,23.68,2021102500,3926,48,2021-10-26 03:14:09,left,None,7.0,...,18.94,Pass Rush,LOLB,NaN,NaN,5.05,-4.28,1.0,1.373062,0


In [488]:
final['end_too_far'] = (final['frameId']>final['ball_snap_frame']+10) &(final['tackle_end_dist']>4)

In [502]:
cluster_data = final.groupby(['uniqueplayId', 'QB_Flip'], as_index=False).agg({
    'end_too_far': lambda x: 1 if x.sum()>=1 else 0,
    'tackle_end_dist': lambda x: (x<1.5).sum()/x.count(),
    'tackle_end_facing': lambda x: x.sum()/x.count(),
    'End_dis': 'sum',
    'End_s':'max'
}).rename(columns={
    'tackle_end_dist':'perc_tackle_lt_1.5',
    'tackle_end_facing':'perc_tackl_end_facing',
    'End_dis':'End_dist_travel',
    'End_s':'Max_end_speed'
})

In [428]:
power = final[(final['uniqueplayId']==2021091902389)&(final['QB_Flip']==0)]

In [429]:
speed = final[(final['uniqueplayId']==20210919023392)&(final['QB_Flip']==0)]

In [509]:
power_clust = cluster_data[(cluster_data['uniqueplayId']==2021091902389)&(cluster_data['QB_Flip']==0)]
speed_clust = cluster_data[(cluster_data['uniqueplayId']==20210919023392)&(cluster_data['QB_Flip']==0)]

In [508]:
power_clust

,uniqueplayId,QB_Flip,end_too_far,perc_tackle_lt_1.5,perc_tackl_end_facing,End_dist_travel,Max_end_speed
519,2021091902389,0.0,0,1.0,0.738095,9.65,3.48


In [510]:
speed_clust

,uniqueplayId,QB_Flip,end_too_far,perc_tackle_lt_1.5,perc_tackl_end_facing,End_dist_travel,Max_end_speed
3551,20210919023392,0.0,0,0.645161,0.451613,12.76,5.58


In [534]:
cluster_final = cluster_data[cluster_data['end_too_far']==0]

In [561]:
cluster_X = cluster_final.drop(
    ['uniqueplayId', 'QB_Flip', 'end_too_far'], axis=1)

In [562]:
kmeans = KMeans(n_clusters=2, random_state=2023).fit(cluster_X)

In [563]:
labs = kmeans.labels_
cluster_final['rushType'] = labs

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_75516/2418565555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_final['rushType'] = labs


In [554]:
random.seed(11902)
rand_ind = random.sample(range(0, len(cluster_final)), 20)

In [564]:
cluster_final.iloc[rand_ind, :]

,uniqueplayId,QB_Flip,end_too_far,perc_tackle_lt_1.5,perc_tackl_end_facing,End_dist_travel,Max_end_speed,rushType
8255,20211017063573,1.0,0,0.666667,0.111111,16.01,5.92,1
8295,20211017071627,0.0,0,0.857143,0.047619,10.71,4.48,1
3327,20210916001343,1.0,0,0.680000,0.120000,5.31,3.66,0
8332,20211017073078,1.0,0,0.620690,0.793103,9.15,4.42,0
7919,20211017012947,0.0,0,0.750000,0.222222,6.46,2.74,0
4911,20210926063298,1.0,0,0.410256,0.256410,5.59,2.26,0
2683,20210912053805,1.0,0,0.545455,0.333333,13.40,5.40,1
1639,2021101400675,0.0,0,0.633333,0.066667,11.49,6.06,1
5293,20210926112428,0.0,0,0.882353,0.588235,7.19,2.92,0
6692,20211010001051,0.0,0,0.878049,0.585366,6.80,2.04,0


In [565]:
cluster_final[cluster_final['uniqueplayId']==20211017063573]

,uniqueplayId,QB_Flip,end_too_far,perc_tackle_lt_1.5,perc_tackl_end_facing,End_dist_travel,Max_end_speed,rushType
8254,20211017063573,0.0,0,0.527778,0.666667,12.01,5.53,1
8255,20211017063573,1.0,0,0.666667,0.111111,16.01,5.92,1


In [558]:
final[final['uniqueplayId']==20211017063573]

,uniqueplayId,football_x,football_y,gameId,playId,frameId,time,playDirection,event,ball_snap_frame,...,End_pff_role,End_pff_positionLinedUp,End_pff_nflIdBlockedPlayer,End_pff_blockType,End_new_x,End_new_y,End_Flip,tackle_end_dist,tackle_end_facing,end_too_far
138885,20211017063573,67.83,29.53,2021101706,3573,6,2021-10-17 19:38:22,right,ball_snap,6.0,...,Pass Rush,SCBR,NaN,NaN,3.33,-8.21,0.0,5.417758,0,False
138886,20211017063573,67.83,29.53,2021101706,3573,7,2021-10-17 19:38:22,right,None,6.0,...,Pass Rush,SCBR,NaN,NaN,3.21,-8.10,0.0,5.230468,0,False
138887,20211017063573,67.83,29.53,2021101706,3573,8,2021-10-17 19:38:22,right,None,6.0,...,Pass Rush,SCBR,NaN,NaN,3.03,-7.97,0.0,4.994437,0,False
138888,20211017063573,67.83,29.53,2021101706,3573,9,2021-10-17 19:38:22,right,None,6.0,...,Pass Rush,SCBR,NaN,NaN,2.78,-7.82,0.0,4.704147,0,False
138889,20211017063573,67.83,29.53,2021101706,3573,10,2021-10-17 19:38:22,right,None,6.0,...,Pass Rush,SCBR,NaN,NaN,2.50,-7.64,0.0,4.385487,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138297,20211017063573,67.83,29.53,2021101706,3573,37,2021-10-17 19:38:25,right,autoevent_passforward,6.0,...,Pass Rush,LOLB,NaN,NaN,9.71,1.02,1.0,1.027862,0,False
138298,20211017063573,67.83,29.53,2021101706,3573,38,2021-10-17 19:38:25,right,None,6.0,...,Pass Rush,LOLB,NaN,NaN,9.65,1.46,1.0,1.095901,0,False
138299,20211017063573,67.83,29.53,2021101706,3573,39,2021-10-17 19:38:25,right,None,6.0,...,Pass Rush,LOLB,NaN,NaN,9.57,1.88,1.0,1.208015,0,False
138300,20211017063573,67.83,29.53,2021101706,3573,40,2021-10-17 19:38:25,right,None,6.0,...,Pass Rush,LOLB,NaN,NaN,9.45,2.27,1.0,1.329436,0,False


In [527]:
kmeans.cluster_centers_

array([[ 0.63966031,  0.44171404,  7.35721035,  3.92475923],
       [ 0.59972663,  0.35666815, 11.93896904,  5.00051974]])

In [531]:
kmeans.get_params()

{'algorithm': 'lloyd',
 'copy_x': True,
 'init': 'k-means++',
 'max_iter': 300,
 'n_clusters': 2,
 'n_init': 10,
 'random_state': 2023,
 'tol': 0.0001,
 'verbose': 0}